# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 3*. Формат JSON и его обработка в рамках блока кода JavaScript: продолжение

Импортируем стандартный набор инструментов – модуль `requests` и функцию `BeautifulSoup`, а также модуль `re` для использования регулярных выражений, модуль `json` для обработки JSON-строк, библиотеку `pandas` для работы с датафреймом.

In [ ]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

Совсем недавно у нас было домашнее задание на парсинг страницы фильма «Не покидай...» с сайта www.kino-teatr.ru. Сайт некоммерческий, довольно дружелюбный, позволяет свободно выгружать информацию (но уже не всегда!).

У него есть одна особенность: число лайков и дизлайков, поставленных актерам пользователями, загружается на страницу динамически, то есть автоматически «подтягивается» с сервера при загрузке страницы в определенный момент времени. На практике это выливается в то, что найти нужную информацию по тэгам просто невозможно, ее нет в основном коде HTML. Как быть? Понять, как выглядит запрос данных, который отправляется на сервер, и выяснить, где хранятся нужные нам данные. Мы рассмотрим несложный случай, когда сайт забирает информацию из строки JSON, которая находится на странице, но внутри кода, написанного на JavaScript. Такое можно встретить на страницах с результатами каких-нибудь игр или на сайтах, посвященных динамике цен или курсу валют (другой вопрос, что не всегда JSON прямо так явно находится в том же файле, где и код HTML).

Так как ранее мы обсуждали довольно базовый парсинг, в домашнем задании не требовалось собирать число лайков и дизлайков, эти значения были даны в виде готового списка словарей. Обновим задачу – теперь нам нужно собрать только id актеров и информацию о числе голосов за и против.

Начало работы стандартное – загружаем код HTML страницы по ссылке и преобразуем его в объект BeautifulSoup:

In [ ]:
# ссылка на копию страницы во избежание блокировок

page = requests.get("https://raw.githubusercontent.com/allatambov/WebScrape25/refs/heads/main/titr.html")
soup = BeautifulSoup(page.text)

Просто найти на странице кнопки красного и зеленого цвета и забрать с них текст не получится. Поэтому для этого на нужно найти код JavaScript, где есть записи с числами `plus` и `minus` с привязкой к id актеров. Код JavaScript, если он не вынесен в отдельный файл, заключается в тэги `<script>`. Если мы внимательно изучим исходный код страницы, мы заметим, что нужный нам блок кода имеет атрибут `type` со значением `text/javascript`, и среди таких блоков он находится на 6-ом месте:

In [ ]:
# выполняем поиск и извлекаем текст – код JavaScript в виде строки

text = soup.find_all("script", {"type" : "text/javascript"})[5].next
#print(text)

In [ ]:
for_votes = text.split("function print_all_role_rating_buttons ()")[1]
print(for_votes)

Хотя с синтаксисом JavaScript знакомы немногие, не очень сложно догадаться, что означают записи такого вида:

    print_role_rating_buttons({ id:"16800", plus:"173", minus:"4", voted:"" });

Такая строка кода активирует функцию print_role_rating_buttons() – применяет ее к такому набору данных и наносит на кнопки, соответствующие актеру с id 16800 значения 173 (зеленая кнопка) и 4 (красная кнопка).

*Примечание 1.* Синхронизация не происходит мгновенно, актуальное значение 173 подставляется на кнопку не сразу, там вполне может стоять значение 172, а мы, благодаря значениям из JavaScript, обладаем более актуальной информацией. Это стоит иметь в виду при выгрузке информации, когда мы сверяем полученные результаты и то, что видим на экране.

*Примечание 2.* Значение `voted` в коде выше является непустым, в случае, если пользователь зарегистрирован и оценивал игру актера (по крайней мере, какое-то время после голосования значение в `voted` держится).

Объект `for_votes` – это обычная строка, объект типа `string`. Поэтому с помощью регулярных выражений мы сможем найти в ней записи в фигурных скобках, затем сделать из них словари, а из списка словарей с единообразными ключами собрать датафрейм!

### Задача 1

Найдите все подстроки, соответствующие наборам символов, заключенных в фигурные скобки – чтобы из них можно было собрать словари. Сохраните список строк в переменную `votes_str`.

In [ ]:
### YOUR CODE HERE ###

Выберем один элемент списка и изучим его:

In [ ]:
v = votes_str[0]
print(v)

На первый взгляд, этот элемент представляет собой полноценный словарь, просто заключенный в кавычки. Однако есть проблема: чтобы переделать строку в словарь, нужна строка, соответствующая формату JSON, а здесь не хватает кавычек вокруг ключей. Как эту проблему решить? Найти по какому-то паттерну эти ключи и доклеить вокруг них кавычки с помощью функции `sub()` из модуля `re` для «умной» замены (не обычная замена, так как мы не конкретный набор символов заменяем на другой, а ищем совпадения по некоторому общему шаблону и их изменяем).

Поиск будет простой – в строке с «неправильным» словарем содержатся либо цифры, либо буквы, либо пробелы с запятыми и знаками препинания, а нам нужны последовательности *из одной и более букв*. 

### Задача 2

Напишите регулярное выражение, которое в строке `v` найдет все слова из букв.

In [ ]:
### YOUR CODE HERE ###

Напишем функцию `add_quotes()`, которая принимает на вход строку `x`, доклеивает к ней кавычки и убирает пробелы. Нам нужно преобразовать строку `x` в группу – объект из регулярных выражений – через `+` доклеить кавычки и на всякий случай убрать лишние пробелы:

In [ ]:
# x.group(): функция sub() из re для умной замены
# работает не со строками, а с объектом специального типа – группа символов

def add_quotes(x):
    g = '"' + x.group() + '"'
    return g.strip()

Применяем функцию `sub` к одной строке `v` и проверяем:

In [ ]:
# аргументы:
# шаблон, по которому ищем, что заменять
# функция, которая выполняет преобразование – доклеивает к результатм поиска кавычки
# строка, по который выполняем поиск

re.sub("[a-z]+", add_quotes, v)

### Задача 3

Примените операцию выше ко всем строкам в `votes_str` и получите список валидных JSON-строк. Используя модуль `json`, превратите список JSON-строк в список питоновских словарей. Создайте на его основе датафрейм pandas.

In [ ]:
### YOUR CODE HERE ###